# The Weighted Ensemble Method

## Sodium Chloride Association Kinetics, with Crossflow

This Notebook illustrates moving a WE workflow from a workstation/laptop to an HPC system. It assumes you have already run through the basic NaCl example.

Since WE worflows typicallty involve running large numbers of short MD simulations each of which is independent of the others, a HPC platform is very useful - as long as a better method can be found for running each simulation than just submitting it to the HPC scheduler.

The combination of `Crossflow` and `dask.distributed` can provide this. We don't have time to go into the details here, but basically `dask.distributed` provides a mechanism to start a personal 'cluster' within an HPC system, to which short jobs - tasks - can be submitted directly, bypassing the usual scheduler. `Crossflow` works with `dask.distributed` to create the individual tasks that run MD simulations on the cluster, and manages the transfer of input and output files around the cluster system.

Here we will use `dask.distributed` not to make a cluster on a HPC system, but to create a 'mini-cluster' of just one worker process, running in the background on your current laptop/desktop. In addition, we will use a fast 'fake" MD application in place of a real compute-intensive MD code. 

Porting this workflow to a real HPC system will involve little more than swapping out the code that creates the mini-cluster for code to create the genuine HPC cluster, and swapping out the code for the fake MD application for that for a real one. Each of these will only require changing a few lines of code (see later).

### Part 0: Install WElib (if not done already):

In [ ]:
!pip install git+http://github.com/CharlieLaughton/WElib.git

### Part 1: FakeMD
Crossflow allows you to write functions that make MD codes like Gromacs or Amber accessible from Python in much the same way as you use OpenMM. For simplicity we will illustrate this using a fake MD code, so there is no requirement for you to have a "proper" MD code installed.

`FakeMD` pretends to be a simple MD code that you would normally run from the command line:

    fake_md -c starting_coordinates -p topology -n nsteps -r final_coordinates
    
where:
* `starting_coordinates` is a Gromacs .gro file or an Amber .ncrst file, or a NAMD .pdb file or suchlike,
* `topology` is a Gromacs topology file or an Amber prmtop file or a NAMD psf file or suchlike,
* `nsteps` is the number of MD steps to run, and
* `final_coordinates` is a Gromacs .gro file or an Amber .ncrst file, or a NAMD .pdb file or suchlike.

The MD done by `fake_md` is entirely bogus - the coordinates are just shifted around a bit randomly - but it can be used to quickly test that a workflow is working properly before its swapped out for a genuine MD code.

Begin by (outside this notebook, probably) putting a copy of the `fake_md` script in a directory that is in your **PATH**. Make sure the permissions are set to make it executable. Then check `fake_md` is in your path:

In [ ]:
! which fake_md

Now we import elements of the crossflow library we will need to turn `fake_md` from something that is run from the command line into something accessible directly from python:

In [ ]:
from crossflow.tasks import SubprocessTask
from crossflow.clients import Client
from crossflow.filehandling import FileHandler
import time
import mdtraj as mdt

Now we create a task to run `fake_md`:

In [ ]:
fake_md = SubprocessTask('fake_md -c input.ncrst -p input.prmtop -n 500 -r output.ncrst')
fake_md.set_inputs(['input.ncrst', 'input.prmtop'])
fake_md.set_outputs(['output.ncrst'])
fh = FileHandler()
inpcrd = fh.load('nacl_unbound.ncrst')
prmtop = fh.load('nacl.parm7')
fake_md.set_constant('input.prmtop', prmtop)

Now we start a single-worker `cluster` on the local machine, and start a `client` as a portal to send jobs to it:

In [ ]:
from dask.distributed import LocalCluster
cluster = LocalCluster(n_workers=1)
client = Client(cluster)

Check to see that the crossflow task can be run via the client. Error messages point to some troubleshooting being required...

In [ ]:
final_crds = client.submit(fake_md, inpcrd)
print(final_crds.status)
time.sleep(5)
print(final_crds.status)

### Part 2: Building the WE workflow
Now we import WElib and other utilities that will be useful. Many are the same as those used for the simple double well potential example, but we have a crossflow-compatible version of the `Stepper` and `ProgressCoordinator`:

In [ ]:
import mdtraj as mdt
import numpy as np
import time
from WElib import Walker, FunctionProgressCoordinator, Recycler, StaticBinner, SplitMerger, Recorder
from WElib.crossflow import CrossflowFunctionStepper

Create some walkers, each begins in the initial, dissociated, state:

In [ ]:
initial_state = inpcrd

n_reps = 5
walkers = [Walker(initial_state, 1.0/n_reps) for i in range(n_reps)]
for w in walkers:
    print(w)

The progress coordinate will be the distance between the sodium and chloride ion. We create a function that, given a state (in this scenario, the restart coordinates file), uses MDTraj to calculate this distance.

In [ ]:
def pc_func(state, topology):
    t = mdt.load(state, top=topology)
    na_atom = 0 # index of the sodium atom in the system
    cl_atom = 1 # index of the chloride ion in the system
    r = mdt.compute_distances(t, [[na_atom, cl_atom]])[0][0]
    return r

progress_coordinator = FunctionProgressCoordinator(pc_func, prmtop)
walkers = progress_coordinator.run(walkers)
for w in walkers:
    print(w)

We use the same bin boundaries as in the WESTPA tutorials. Notice these are closer-spaced at shorter distances, as the solvation shells get "stiffer":

In [ ]:
binner = StaticBinner([0, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.45, 0.5, 
                 0.55, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5])
walkers = binner.run(walkers)
for w in walkers:
    print(w)

We will recycle walkers when the Na-Cl distance falls below 0.26 nm. As the progress coordinate is something that gets smaller as we move towards the target state, this is a "retrograde" coordinate:

In [ ]:
recycler = Recycler(0.26, retrograde=True)
walkers = recycler.run(walkers)
for w in walkers:
    print(w)
print('recycled flux = ',recycler.flux)

The SplitMerger is just the same as that used for the DWP example. We create it and run it, even though we know that at this time it will have nothing to do:

In [ ]:
splitmerger = SplitMerger(n_reps)
walkers = splitmerger.run(walkers)
for w in walkers:
    print(w)

We create a `stepper` from the `CrossflowFunctionStepper` class. The arguments are the Dask/Crosasflow `client` that connectes us to the cluster, the Crossflow `fake_md` task, and the extra arguments that task function takes (just the prmtop file):

In [ ]:
stepper = CrossflowFunctionStepper(client, fake_md)

Then we will apply the stepper. Because we are using "fake_md", it should run fairly fast, whatever the spec of your laptop/desktop:

In [ ]:
start_time = time.time()
new_walkers = stepper.run(walkers) # this is where the MD happens
end_time = time.time()
print(f'{len(walkers)} simulations completed in {end_time-start_time:6.1f} seconds')

Let's see where those MD steps have moved each walker to:

In [ ]:
new_walkers = progress_coordinator.run(new_walkers)
new_walkers = binner.run(new_walkers)
new_walkers = recycler.run(new_walkers)
print('recycled flux = ', recycler.flux)
for w in new_walkers:
    print(w)

Apply the SplitMerger to the list of walkers:

In [ ]:
new_walkers = splitmerger.run(new_walkers)
for w in new_walkers:
    print(w)

### Part 3: Iterating the WE workflow
OK, that's all the components in place, they have been tested individually and seem to be behaving. Time to run a few cycles:

In [ ]:
n_cycles=10
print(' cycle    n_walkers   left-most bin  right-most bin   flux')
for i in range(n_cycles):
    new_walkers = stepper.run(new_walkers)
    new_walkers = progress_coordinator.run(new_walkers)
    new_walkers = binner.run(new_walkers)
    new_walkers = recycler.run(new_walkers)
    if recycler.flux > 0.0:
        new_walkers = progress_coordinator.run(new_walkers)
        new_walkers = binner.run(new_walkers)
    new_walkers = splitmerger.run(new_walkers)
    occupied_bins = list(binner.bin_weights.keys())
    print(f' {i:3d} {len(new_walkers):10d} {min(occupied_bins):12d} {max(occupied_bins):14d} {recycler.flux:20.8f}')

(You can ignore any warnings about time taken for garbage collection - they are due to the low CPU overhead of the fake MD code and will go away when a real MD code is used.)

#### Porting to a real HPC system

As mentioned above, if we were running this notebook on a HPC system, e.g. ARCHER2, we could now swap out the 'mini-cluster' for a real one, and the fake MD code for a real one. Here we outline the process.

##### From fake MD to Gromacs
First we would neeed to convert the Amber files `nacl_unbound.ncrst` and `nacl.parm7` to Gromacs equivalents - Amber provides some utilities to do this. 

Then we create a `Crossflow` task to run both `grompp` and `mdrun` steps of a Gromacs simulation. Details might vary, but it will be something like this:

    rungmx = SubprocessTask('gmx grompp -f x.mdp -c x.gro -p x.top -o x.tpr -maxwarn 1; srun --distribution=block:block --hint=nomultithread gmx_mpi mdrun -s x.tpr -c y.gro')
    rungmx.set_inputs(['x.gro', 'x.top'])
    rungmx.set_outputs(['y.gro'])
    fh = FileHandler()
    inpcrd = fh.load('nacl_unbound.gro')
    top = fh.load('nacl.top')
    rungmx.set_constant('x.top', top)
    
##### From mini-cluster to HPC cluster
Instructions on how to create a `dask.distributed` cluster on ARCHER2 are included in the user guide - see the section about `dask-jobqueue` [here](https://docs.archer2.ac.uk/user-guide/python/). Once you have got your virtual environment set up, etc., then to create a suitable `cluster` you would amend your Jupyter notebook something like this:

        cluster = SLURMCluster(cores=1,
                       job_cpu=1,
                       processes=1,
                       memory='256GB',
                       queue='standard',
                       header_skip=['-n ', '--mem'],
                       interface='hsn0',
                       job_extra_directives=['--nodes=1',
                           '--qos="standard"',
                           '--tasks-per-node=128'],
                       python='python',
                       project='xxxxxx',  # put your account code in here
                       walltime="01:00:00",
                       shebang="#!/bin/bash --login",
                       local_directory='$PWD',
                       job_script_prologue=['module load gromacs',
                          'export PYTHONUSERBASE=/some/work/directory/path/.local',
                          'export PATH=$PYTHONUSERBASE/bin:$PATH',
                          'export PYTHONPATH=$PYTHONUSERBASE/lib/<python_version>/site-packages:$PYTHONPATH',
                          'export OMP_NUM_THREADS=1',
                          'source /path/to/virtual/environment/bin/activate'])
    
        print('scaling cluster...')
        cluster.scale(n_workers) # the number of worker processes - each will be one ARCHER2 node
        client = Client(cluster)
        

##### Create the new stepper

Now you have a new `Crossflow` task to run a Gromacs simulation on Archer2, and the specification for an Archer2 cluster, the two can be used to create the WE `Stepper`:

    stepper = CrossflowFunctionStepper(client, rungmx)
    
    
And you should be good to go.

Obviously in reality you are likely to run WE jobs on Archer2 from Python scripts rather than interactively through Jupyter notebooks, but the essential code will be the same.